<!-- dom:TITLE: STAT 5102 Regression in Practice Final Project -->
# <center>Generalised Linear Model for Estimating Mortality Rate Using Development Indicators</center>


<!-- dom:AUTHOR: CHAN, Yiu Fung Frankie, 1155010561 -->
<!-- Author: -->  
**<h5><center>Chan, Yiu Fung Frankie,  1155010561</center></h5>**  
<!-- dom:AUTHOR: LI Wai Lan, 1155118254 -->
<!-- Author: -->  
**<h5><center>LI Wai Lan,  1155118254</center></h5>** 
<!-- dom:AUTHOR: WONG, Tsz Wing, 1004666311 -->
<!-- Author: -->  
**<h5><center>Wong, Tsz Wing,  1004666311</center></h5>**
<!-- dom:AUTHOR: WONG, Yiu Chung, 1155017920 -->
<!-- Author: -->  
**<h5><center>WONG, Yiu Chung,  1155017920</center></h5>** 


Date: **Nov 18, 2018**

In this study, we investigate the dataset obtained from The World Bank: World Development Indicators. A Generalised Linear Model is used to perform a prediction analysis on the response variable: Mortality rate, using ???(number) predictor variables.

In [189]:
from sklearn.preprocessing import PolynomialFeatures
import sklearn
import pandas as pd
import numpy as np
import os
import pycountry #pycountry provides the ISO databases for the standards:

In [190]:
#change working directory to 5102 project folder
os.chdir(r"/Users/chung/Google Drive/2018R1 Regression in Practice (STAT5102)/5012 Project")

#read project_data.sas7bdat
world_bank = pd.read_sas(r"project_data.sas7bdat", encoding="utf-8")

In [193]:
#Drop the column Year and YearCode
world_bank = world_bank.drop(["Year", "YearCode"], axis=1)

#remove columns with more than 20% Nan
nrow = world_bank.shape[0]
na_portion = world_bank.isna().sum() / nrow
world_bank = world_bank.loc[:, na_portion < .2]

#remove case if Mortality rate, under-5 (per 1,000 live births) is Nan
world_bank = world_bank.loc[world_bank["Mortality rate, under-5 (per 1,0"].notnull(),:] 

In [195]:
#import functions for converting country code into continents codes
from pycountry_convert import (
    map_countries,
    country_alpha2_to_country_name,
    country_name_to_country_alpha2,
    country_alpha3_to_country_alpha2,
    country_alpha2_to_continent_code,
    COUNTRY_NAME_FORMAT_UPPER
)

#convert country code into continents codes
continents = []
fail_codes = []
for country_code in world_bank["Country Code"]:
    try:
        alpha2 = country_alpha3_to_country_alpha2(country_code)
        continent = country_alpha2_to_continent_code(alpha2)
    except KeyError:
        fail_codes.append(country_code)
        continents.append("999") #temporary place holder
    else:    
        continents.append(continent)

In [196]:
#see what's in the continents list variable
set(continents)

{'999', 'AF', 'AS', 'EU', 'NA', 'OC', 'SA'}

In [197]:
#insert continents into the world_bank dataframe
#insert as column 2, named "Continent"
world_bank.insert(2, "Continent", continents)

In [210]:
#manually impute fail codes
fail_codes_continents = ["EU", "AF", "EU", "AS", "AS"] #in this order. I checked manually.
fail_code_dict = dict(zip(fail_codes, fail_codes_continents))

#for columns with Country Code that matches elements in the fail_codes variable, 
#replace their Continent value (whcih should be 999 for now) with the elements in fail_codes_continents
for code, continent in fail_code_dict.items():
    world_bank.loc[world_bank["Country Code"] == code, "Continent"] = continent

In [200]:
#export world_bank to excel file to share with groupmates (18 Nov 2018)
writer = pd.ExcelWriter('world_bank.xlsx')
world_bank.to_excel(writer, 'Sheet1')
writer.save()

In [223]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(world_bank)

,Country Name,Country Code,Continent,Adjusted savings: education expe,Adolescent fertility rate (birth,Age dependency ratio (% of worki,"Age dependency ratio, young (% o",Agricultural land (% of land are,"Birth rate, crude (per 1,000 peo","Fertility rate, total (births pe",GDP (constant 2005 US$),GDP growth (annual %),"GDP per capita, PPP (constant 20",Health expenditure per capita (c,"Health expenditure, public (% of","Health expenditure, public _0001","Inflation, GDP deflator (annual","Life expectancy at birth, female","Life expectancy at birth, male (","Life expectancy at birth, total","Mortality rate, infant (per 1,00","Mortality rate, under-5 (per 1,0","Mortality rate, under-5, female","Mortality rate, under-5, male (p",Population growth (annual %),"Immunization, DPT (% of children","Immunization, measles (% of chil"
0,Afghanistan,AFG,AS,NaN,99.095600,103.119626,98.722026,58.123668,37.952000,5.6590,1.024325e+10,8.433290,1183.045192,44.187558,2.352934,3.858997,9.437795,60.877000,58.384000,59.600098,74.3,103.7,100.4,106.7,2.458417,66.0,62.0
1,Albania,ALB,EU,2.842804,16.994600,49.098446,34.036210,43.843066,12.607000,1.7410,1.072515e+10,3.500000,7791.755363,220.228562,2.532118,8.533154,3.459343,80.149000,73.959000,76.978512,16.1,18.1,16.3,19.7,-0.033072,99.0,99.0
2,Algeria,DZA,AF,4.467196,9.894599,46.741610,39.796430,17.370914,24.514000,2.8170,1.166263e+11,3.600000,7249.230810,198.155638,3.465480,8.965406,16.119976,72.229000,69.081000,70.616610,18.4,21.5,19.0,23.8,1.850438,95.0,95.0
4,Andorra,ADO,EU,2.889150,NaN,NaN,NaN,42.765957,9.800000,1.2200,NaN,NaN,NaN,2957.555642,5.288827,21.251895,NaN,NaN,NaN,NaN,2.9,3.5,3.1,3.8,-0.960625,99.0,99.0
5,Angola,AGO,AF,3.516770,178.994000,100.915476,96.050884,46.835646,46.327000,6.2180,5.037116e+10,3.407644,5048.113176,146.109850,2.061268,6.300767,22.393924,52.138000,49.241000,50.654171,104.6,173.1,164.5,181.1,3.235950,91.0,93.0
6,Antigua and Barbuda,ATG,NaN,2.159410,51.755000,50.300659,39.428660,20.454545,16.966000,2.1300,1.009446e+09,-7.738048,17610.216110,771.382513,4.196069,18.668552,-0.010767,77.795000,72.990000,75.333902,9.8,10.6,9.4,11.7,1.075310,98.0,98.0
7,Argentina,ARG,SA,6.009790,55.372400,55.033758,38.566710,53.611114,17.188000,2.2150,NaN,NaN,NaN,759.299433,5.336253,17.688600,NaN,79.462000,72.046000,75.663561,13.3,14.8,13.3,16.3,0.872126,94.0,98.0
8,Armenia,ARM,AS,2.219690,27.481600,45.059361,29.758869,60.919944,14.140000,1.7380,7.611466e+09,2.200000,6582.659928,133.785599,1.811066,6.421649,7.768715,77.657000,70.946000,74.219659,16.2,18.2,16.1,20.2,-0.157056,94.0,97.0
10,Australia,AUS,OC,4.775350,13.645400,47.909932,28.019034,51.882900,13.120148,1.8700,7.988266e+11,2.087576,34620.923355,5173.502443,6.190147,16.758304,0.886791,84.000000,79.500000,81.695122,4.2,5.0,4.4,5.6,1.306922,92.0,94.0
11,Austria,AUT,EU,5.288180,6.926000,48.328841,21.881827,38.405999,9.400000,1.4400,3.255503e+11,1.769169,35201.155150,4964.154032,8.364864,15.908260,1.440785,83.200000,77.700000,80.382927,3.5,4.2,3.7,4.6,0.292402,83.0,76.0


In [ ]:
#Want to use the PolynomialFeatures function to generate quadratic terms for Polynomial regression

In [141]:
#test PolynomialFeatures function
data = pd.DataFrame.from_dict({
    'x': np.random.randint(low=1, high=10, size=5),
    'y': np.random.randint(low=-1, high=1, size=5),
})

p = PolynomialFeatures(degree=2).fit(data)
p.get_feature_names(data.columns)

['1', 'x', 'y', 'x^2', 'x y', 'y^2']

In [159]:
#have to deal with continent column.
quadratic_terms = PolynomialFeatures(degree=2).fit(world_bank.iloc[:, 3:])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').